In [2]:
import uuid
import pandas as pd
from arango import ArangoClient

In [3]:
raw_df = pd.read_csv('./data/raw.csv')
raw_df.head()

,country_of_destination,country_of_origin,hs6_code,receiver_address,receiver_id,receiver_name,sender_address,sender_id,sender_name,tier,transaction_date
0,CA,US,321410,"95 SUNRAY STREET, WHITBY, ON, L1N9C9, CANADA T...",4bd999c0cb4346ce047f99a692ec3191,Stonhard Ltd,"60 MARKET SAUQRE, BELIZE CITY, BELIZE",52c8642c34649eafe2d044eee3d884e1,Global Fire Protection,t1->t0,11/30/18
1,NaN,NaN,320890,China,8b1f537a8eeca839cff8a99e08b35690,CARBOLINE DALIAN PAINT COMPANY LTD,"Plot O 356 &357, Sidco Indl Estate Ambattur Ch...",831a31a1466f0ace3eb20b52d4575f92,Carboline (India) Private Limited,t1->t0,5/2/18
2,US,IN,390940,United States,1670c05b29762f5d4ab8980d267bd482,Rust-Oleum Llc,"204, Monarch Chambers, Marol Maroshi Road, Mum...",f580bc1756d06768c94634b0332e1871,Paladin Paints & Chemicals Private Limited,t1->t0,5/2/17
3,US,CN,731100,OF RPM WOOD FINISHES GROUP 3190 HICKORY BLVD H...,210f645e15d75833b25090f2ae509eda,Mohawk Finishing Products Inc,"YUNG-SHI BLVD, SHI-WAN TOWN, PO-LO COUNTY HUI-...",149081be00548b006dc38a88264eae32,Alpha Pacific Group Pte. Ltd.,t1->t0,5/8/16
4,CH,IN,250590,Switzerland,7150428f8927c86ac11c64c1bb9b0285,Vandex International AG,FLOWCRETE INDIA PRIVATE LIMITEDNEW NO.36 OLD N...,f4864ac3d5d716cc586d60afe8a403ef,TREMCO CPG INDIA PRIVATE LIMITED,t1->t0,5/26/17


In [4]:
address_df = pd.read_csv('./data/address.csv')

In [5]:
address_df

,organization_id,address,country
0,52c8642c34649eafe2d044eee3d884e1,"60 MARKET SAUQRE, BELIZE CITY, BELIZE",US
1,831a31a1466f0ace3eb20b52d4575f92,"Plot O 356 &357, Sidco Indl Estate Ambattur Ch...",US
2,f580bc1756d06768c94634b0332e1871,"204, Monarch Chambers, Marol Maroshi Road, Mum...",IN
3,149081be00548b006dc38a88264eae32,"YUNG-SHI BLVD, SHI-WAN TOWN, PO-LO COUNTY HUI-...",CN
4,f4864ac3d5d716cc586d60afe8a403ef,FLOWCRETE INDIA PRIVATE LIMITEDNEW NO.36 OLD N...,IN
...,...,...,...
2541,00750bb980757fd8535d73b90d6608c8,4518 HAMILTON AVENUE CLEVELAND OHIO 44114 USA ...,US
2542,2c398f54dacfc67b9fbe414565285dba,19320 REDWOOD RD. 44110-2799 CLEVELAND CLEVELAND,US
2543,2c398f54dacfc67b9fbe414565285dba,"19218 REDWOOD ROAD CLEVELAND, OH CLEVELANDOH ...",US
2544,1b4334d45be8824a1f6b071a9a2f75a8,United States,US


In [ ]:
# merged_df = pd.merge(raw_df, address_df, left_on=["receiver_id", 

In [6]:
raw_df[['receiver_id','sender_id', 'transaction_date']]

,receiver_id,sender_id,transaction_date
0,4bd999c0cb4346ce047f99a692ec3191,52c8642c34649eafe2d044eee3d884e1,11/30/18
1,8b1f537a8eeca839cff8a99e08b35690,831a31a1466f0ace3eb20b52d4575f92,5/2/18
2,1670c05b29762f5d4ab8980d267bd482,f580bc1756d06768c94634b0332e1871,5/2/17
3,210f645e15d75833b25090f2ae509eda,149081be00548b006dc38a88264eae32,5/8/16
4,7150428f8927c86ac11c64c1bb9b0285,f4864ac3d5d716cc586d60afe8a403ef,5/26/17
...,...,...,...
99879,66ebf2a3aa71c90b4df3fc93590e22db,25fe0d54401de2e86f6ebb4828648e0e,5/10/19
99880,66ebf2a3aa71c90b4df3fc93590e22db,25fe0d54401de2e86f6ebb4828648e0e,5/10/19
99881,66ebf2a3aa71c90b4df3fc93590e22db,25fe0d54401de2e86f6ebb4828648e0e,5/10/19
99882,66ebf2a3aa71c90b4df3fc93590e22db,25fe0d54401de2e86f6ebb4828648e0e,5/10/19


In [7]:
raw_df.rename(columns={'transaction_date':'date'}, inplace=True)
raw_df['_from'] = 'organizations/' + raw_df['sender_id']
raw_df['_to'] = 'organizations/' + raw_df['receiver_id']
raw_df['_key'] = [uuid.uuid4().hex for _ in range(len(raw_df))]

In [8]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [9]:
# db.delete_collection('transactions')
if not db.has_collection('transactions'):
    transactions = db.create_collection('transactions', edge=True)
else:
    transactions = db.collection('transactions')

In [10]:
transactions_list = raw_df[['_key', '_from', '_to', 'date']].to_dict(orient='records')

In [41]:
transactions.insert_many(transactions_list, overwrite_mode="replace", silent=True)

True

In [36]:
assert transactions.count() == raw_df.shape[0]

In [42]:
transactions.count()

99884